In [1]:
import pandas as pd
import numpy as np
np.seterr(invalid='ignore')

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
%matplotlib inline

matplotlib.rcParams['figure.figsize'] = (12, 10)
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']


import os, datetime
import tempfile

import seaborn as sns

os.environ["CUDA_VISIBLE_DEVICES"]="4"
#import tensorflow as tf
#from tensorflow import keras



from sklearn.mixture import GaussianMixture

from sklearn.metrics import roc_auc_score, average_precision_score, brier_score_loss

#path = '/data/datasets/topcat/data/csv_data/'
path = '/data/datasets/topcat/data/sas_data/'
pathOutcomes = '/data/datasets/topcat/data/Outcomes/'

from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

skf = StratifiedKFold(n_splits=5, shuffle=False)

from collections import defaultdict

from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier, BalancedBaggingClassifier, RUSBoostClassifier, EasyEnsembleClassifier
from sklearn.calibration import CalibratedClassifierCV
from califorest.califorest import CaliForest

In [2]:
def brierskillscore(y_pred, y_true):
    cls_num_list = np.unique(y_true, return_counts=True)[1]
    ratio = float(cls_num_list[1]) / np.sum(cls_num_list)
    y_true, y_pred = y_true.reshape(1,-1).squeeze(), y_pred.reshape(1,-1).squeeze()
    y_pred_sig = 1/(1 + np.exp(-y_pred))
    predictions_ref = [ratio] * len(y_pred)
    BS = brier_score_loss(y_true, y_pred_sig)
    BS_ref = brier_score_loss(y_true, predictions_ref)
    BS_skill = 1 - float(BS)/BS_ref

    print('ratio', ratio)
    print('BS_ref', BS_ref)
    print('BS', BS)
    return BS_skill

In [3]:
outcome_cols = [
    'death', 'cvd_death', 'time_death', 'anyhosp', 'time_anyhosp',
    'hfhosp', 'time_hfhosp', 'abortedca', 'time_abortedca', 'mi',
    'time_mi', 'stroke', 'time_stroke', 'primary_ep', 'time_primary_ep'
]

In [4]:
con_cat_cols = [
    'GLUCOSE_FAST', 'GLUCOSE_RAND', 'CO2_mmolL', 'GLUCOSE_mgdL','WBC_kuL',
    'HCT_p', 'HB_gdL', 'PLT_kuL', 'ALP_UL', 'TBILI_mgdL', 'ALB_gdL'
]

In [5]:
contin_cols = [
    'BNP_VAL', 'age_entry', 'EF', 'visit_dt1_hf', 'chfdc_dt3', 'mi_dt3',
    'stroke_dt3', 'cabg_dt3', 'pci_dt3', 'DM_AGE_YR', 'DM_DUR_YR', 'cigs',
    'SMOKE_YRS', 'QUIT_YRS', 'HEAVY_MIN', 'HEAVY_WK', 'MED_WK', 'MED_MIN',
    'LIGHT_WK', 'LIGHT_MIN', 'metsperweek', 'cooking_salt_score', 'height',
    'weight', 'waistc', 'HR', 'SBP', 'DBP', 'CR_mgdl', 'gfr', 'labs_dt1',
    'NA_mmolL', 'K_mmolL', 'CL_mmolL', 'BUN_mgdL', 'ALT_UL', 'AST_UL',
    'urine_val_mgg', 'QRS_DUR', 'CR_mgdL', 'BMI'
]

In [6]:
priep = pd.read_csv('/data/datasets/topcat/nch/nn_baseline/primary_ep_set.csv', index_col=0)
death = pd.read_csv('/data/datasets/topcat/nch/nn_baseline/death_set.csv'     , index_col=0)
hfhos = pd.read_csv('/data/datasets/topcat/nch/nn_baseline/hfhosp_set.csv'    , index_col=0)

In [7]:
skf = StratifiedKFold(n_splits=5, shuffle=False)

In [8]:
mode = 2

In [9]:
if mode == 1:
    outcome = 'primary_ep'
    outcome_time = 'time_primary_ep'
    df = priep.copy()
elif mode == 2:
    outcome = 'death'
    outcome_time = 'time_death'
    df = death.copy()
elif mode == 3:
    outcome = 'hfhosp'
    outcome_time = 'time_hfhosp'
    df = hfhos.copy()

labels = df[outcome].copy()
complete_labels = labels.copy()

METRICS = [
  keras.metrics.TruePositives(name='tp'),
  keras.metrics.FalsePositives(name='fp'),
  keras.metrics.TrueNegatives(name='tn'),
  keras.metrics.FalseNegatives(name='fn'), 
  keras.metrics.BinaryAccuracy(name='accuracy'),
  keras.metrics.Precision(name='precision'),
  keras.metrics.Recall(name='recall'),
  keras.metrics.AUC(name='auc'),
]

def make_model(metrics = METRICS, output_bias=None, hl_count=1, hl_size=16, dropout=0.5):
        if output_bias is not None:
            output_bias = tf.keras.initializers.Constant(output_bias)
        model = keras.Sequential()
        model.add(keras.layers.Dense(
                hl_size, activation='relu',
                input_shape=(train_data.shape[-1],)))
        model.add(keras.layers.Dropout(dropout))
        
        for i in range(hl_count-1):
            model.add(
                keras.layers.Dense(hl_size, activation='relu'))
            model.add(keras.layers.Dropout(dropout))
        
        model.add(keras.layers.Dense(1, activation='sigmoid',
                               bias_initializer=output_bias)
        )
        model.compile(
            optimizer=keras.optimizers.Adam(lr=1e-3),
            loss=keras.losses.BinaryCrossentropy(),
            metrics=metrics)
        return model

EPOCHS = 2000
BATCH_SIZE = 64

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_auc', 
    verbose=0,
    patience=200,
    mode='max',
    restore_best_weights=True)


pos_idx = np.where(train_labels == 1)[0]

neg_idx = np.where(train_labels == 0)[0]

In [34]:
auc_df = pd.DataFrame(index=range(5))
auprc_df = pd.DataFrame(index=range(5))
bss_df = pd.DataFrame(index=range(5))

for i, (train, test) in enumerate(skf.split(df, labels)):
    print(f'Fold {i}: {datetime.datetime.now().strftime("%I:%M:%S %p")}  ', end='')
    train_data = df.iloc[train].copy()
    test_data = df.iloc[test].copy()
    
    train_labels=labels.iloc[train].copy()
    test_labels=labels.iloc[test].copy()
    
    counts = np.unique(train_labels, return_counts=True)
    weight_for_0 = 1.0 / counts[0]
    weight_for_1 = 1.0 / counts[1]
    class_weight = {0: weight_for_0, 1: weight_for_1}

    weights = len(train_labels)/test_labels.sum()
    glm_weights = pd.Series(data=1, index=train_labels.index)
    glm_weights.loc[train_labels==1] = weights
    
    #remove outcomes
    train_id = train_data['ID'].copy()
    test_id = test_data['ID'].copy()
    
    train_data.drop(columns=outcome_cols+['ID'], inplace=True)
    test_data.drop(columns= outcome_cols+['ID'], inplace=True)
    
    #print(f'Fold {i} Imputation')
    #imp = SimpleImputer(missing_values=np.nan, strategy='mean')
    #train_data.values = imp.fit_transform(train_data)
    #test_data.values  = imp.transform(test_data)
    test_data = test_data.fillna(train_data.mean())
    train_data = train_data.fillna(train_data.mean())
    
    sd_0_cols = train_data.columns[(train_data.std() == 0)]
    train_data.drop(columns=sd_0_cols, inplace=True)
    test_data.drop(columns=sd_0_cols, inplace=True)
    
    cols_to_scale = [foo for foo in con_cat_cols + contin_cols if foo in train_data.columns]
    scaler = StandardScaler()
    train_data.loc[:,cols_to_scale] = scaler.fit_transform(train_data.loc[:,cols_to_scale])
    test_data.loc[:,cols_to_scale]  = scaler.transform(test_data.loc[:,cols_to_scale])
    
    
    ############################
    '''
    xgb = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
    #xgb_cali = CalibratedClassifierCV(base_estimator=xgb, method='isotonic', cv=3)
    #xgb = xgb_cali
    xgb.fit(train_data, train_labels)
    auc = roc_auc_score(test_labels, xgb.predict_proba(test_data)[:,1])
    auprc = average_precision_score(test_labels, xgb.predict_proba(test_data)[:,1])
    bss = brierskillscore(xgb.predict_proba(test_data)[:,1], test_labels.values)
    #print(f'XGB AUC={auc:.3f}\tNNs:')
    #print(f'XGB AUPRC={auprc:.3f}\tNNs:')
    #print(f'XGB BSS={bss:.3f}\tNNs:')
    if i == 0:
        auc_df['xgb'] = np.nan
        auprc_df['xgb'] = np.nan
        bss_df['xgb'] = np.nan
    auc_df['xgb'].iloc[i] = auc
    auprc_df['xgb'].iloc[i] = auprc
    bss_df['xgb'].iloc[i] = bss
    
    ############################
    rf = RandomForestClassifier()
    #rf_cali = CalibratedClassifierCV(base_estimator=rf, method='isotonic', cv=3)
    #rf = rf_cali
    rf.fit(train_data, train_labels)
    auc = roc_auc_score(test_labels, rf.predict_proba(test_data)[:,1])
    auprc = average_precision_score(test_labels, rf.predict_proba(test_data)[:,1])
    bss = brierskillscore(rf.predict_proba(test_data)[:,1], test_labels.values)
    #print(f'RF AUC={auc:.3f}\nNNs:')
    #print(f'RF AUPRC={auprc:.3f}\nNNs:')
    #print(f'RF BSS={bss:.3f}\nNNs:')
    if i == 0:
        auc_df['rf'] = np.nan
        auprc_df['rf'] = np.nan
        bss_df['rf'] = np.nan
    auc_df['rf'].iloc[i] = auc
    auprc_df['rf'].iloc[i] = auprc
    bss_df['rf'].iloc[i] = bss
    
    ############################
    rf_b = RUSBoostClassifier()
    #rf_b_cali = CalibratedClassifierCV(base_estimator=rf_b, method='isotonic', cv=3)
    #rf_b = rf_b_cali
    rf_b.fit(train_data, train_labels)
    auc = roc_auc_score(test_labels, rf_b.predict_proba(test_data)[:,1])
    auprc = average_precision_score(test_labels, rf_b.predict_proba(test_data)[:,1])
    bss = brierskillscore(rf_b.predict_proba(test_data)[:,1], test_labels.values)
    #print(f'RF AUC={auc:.3f}\nNNs:')
    #print(f'RF AUPRC={auprc:.3f}\nNNs:')
    #print(f'RF BSS={bss:.3f}\nNNs:')
    if i == 0:
        auc_df['rf_b'] = np.nan
        auprc_df['rf_b'] = np.nan
        bss_df['rf_b'] = np.nan
    auc_df['rf_b'].iloc[i] = auc
    auprc_df['rf_b'].iloc[i] = auprc
    bss_df['rf_b'].iloc[i] = bss
    '''
    ############################
    califorest = CaliForest()
    califorest.fit(train_data, train_labels)
    auc = roc_auc_score(test_labels, califorest.predict_proba(test_data)[:,1])
    auprc = average_precision_score(test_labels, califorest.predict_proba(test_data)[:,1])
    bss = brierskillscore(califorest.predict_proba(test_data)[:,1], test_labels.values)
    #print(f'RF AUC={auc:.3f}\nNNs:')
    #print(f'RF AUPRC={auprc:.3f}\nNNs:')
    #print(f'RF BSS={bss:.3f}\nNNs:')
    if i == 0:
        auc_df['califorest'] = np.nan
        auprc_df['califorest'] = np.nan
        bss_df['califorest'] = np.nan
    auc_df['califorest'].iloc[i] = auc
    auprc_df['califorest'].iloc[i] = auprc
    bss_df['califorest'].iloc[i] = bss
    
    ############################
    '''
    #for hl_count in [1,2,3,4]:
    for hl_count in [1]:
        print(f'\t{hl_count} HL(s): ', end='')
        #for hl_size in np.linspace(6,60,dtype=int, num=6):
        for hl_size in [6]:
            model = make_model(hl_count=hl_count, hl_size=hl_size)
            name = f'{hl_count}_{hl_size}'
            if name not in auc_df.columns:
                auc_df[name] = np.nan
            
            baseline_history = model.fit(
                train_data.values,
                train_labels.values,
                batch_size=BATCH_SIZE,
                epochs=EPOCHS,
                callbacks = [early_stopping],
                validation_data=(test_data.values, test_labels.values),
                class_weight=class_weight,
                verbose=0
            )
            auc = np.mean(baseline_history.history['val_auc'][-100:])
            run_length = len(baseline_history.history['val_auc'])
            auc_df[name].iloc[i] = auc
            print(f'size{hl_size}=> AUC={auc:.3f} ({str(run_length).rjust(4)} epochs)   ', end='')
        print()
            
    '''    
    #pos = train_labels.sum()
    #total = len(train_labels)
    #neg = total - pos

    

Fold 0: 06:47:03 PM  

/home/grads/g/guangzhou92/miniconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:14: RuntimeWarning: divide by zero encountered in true_divide
  


ratio 0.2889908256880734
BS_ref 0.205475128356199
BS 0.2723203624267018
Fold 1: 06:47:04 PM  

/home/grads/g/guangzhou92/miniconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:14: RuntimeWarning: divide by zero encountered in true_divide
  


ratio 0.2889908256880734
BS_ref 0.205475128356199
BS 0.2729867463235877
Fold 2: 06:47:05 PM  

...

ratio 0.28440366972477066
BS_ref 0.20351822237185416
BS 0.27220696991113524
Fold 3: 06:47:05 PM  

/home/grads/g/guangzhou92/miniconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:14: RuntimeWarning: divide by zero encountered in true_divide
  


ratio 0.2857142857142857
BS_ref 0.20408163265306126
BS 0.2731182754541424
Fold 4: 06:47:06 PM  

/home/grads/g/guangzhou92/miniconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:14: RuntimeWarning: divide by zero encountered in true_divide
  


ratio 0.2857142857142857
BS_ref 0.20408163265306126
BS 0.2773880586968818


auc_df

auprc_df

bss_df

In [35]:
auc_scores = auc_df.T
auc_scores['lower_ci'] = auc_df.T.mean(axis=1) - 1.96*auc_df.T.std(axis=1)/auc_df.T.count(axis=1)
auc_scores['mean'] = auc_df.T.mean(axis=1)
auc_scores['upper_ci'] = auc_df.T.mean(axis=1) + 1.96*auc_df.T.std(axis=1)/auc_df.T.count(axis=1)
#auc_scores.sort_values(by='mean', ascending=False)
auc_scores

,0,1,2,3,4,lower_ci,mean,upper_ci
califorest,0.735177,0.776907,0.758943,0.706191,0.684079,0.717443,0.73226,0.747077


In [36]:
auprc_scores = auprc_df.T
auprc_scores['lower_ci'] = auprc_df.T.mean(axis=1) - 1.96*auprc_df.T.std(axis=1)/auprc_df.T.count(axis=1)
auprc_scores['mean'] = auprc_df.T.mean(axis=1)
auprc_scores['upper_ci'] = auprc_df.T.mean(axis=1) + 1.96*auprc_df.T.std(axis=1)/auprc_df.T.count(axis=1)
#auc_scores.sort_values(by='mean', ascending=False)
auprc_scores

,0,1,2,3,4,lower_ci,mean,upper_ci
califorest,0.508045,0.522168,0.542909,0.466901,0.415518,0.47128,0.491109,0.510937


In [37]:
bss_scores = bss_df.T
bss_scores['lower_ci'] = bss_df.T.mean(axis=1) - 1.96*bss_df.T.std(axis=1)/bss_df.T.count(axis=1)
bss_scores['mean'] = bss_df.T.mean(axis=1)
bss_scores['upper_ci'] = bss_df.T.mean(axis=1) + 1.96*bss_df.T.std(axis=1)/bss_df.T.count(axis=1)
#auc_scores.sort_values(by='mean', ascending=False)
bss_scores

,0,1,2,3,4,lower_ci,mean,upper_ci
califorest,-0.32532,-0.328563,-0.337507,-0.33828,-0.359201,-0.342958,-0.337774,-0.332591


from numba import cuda
cuda.select_device(0)
cuda.close()